In [1]:
import sys
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file

from custom_scripts import database
from custom_scripts import preprocessing

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
sample_number = 1000

In [3]:
flights_query =  f"""
                SELECT * FROM flights ORDER BY RANDOM() LIMIT '{sample_number}';
                """

flights = database.query(flights_query)

In [4]:
flights.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-10-02,UA,UA_CODESHARE,UA,4696,AX,N31131,4696,10627,BIS,...,716.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2019-09-04,HA,HA,HA,226,HA,N490HA,226,12173,HNL,...,100.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2018-07-03,UA,UA_CODESHARE,UA,5798,OO,N119SY,5798,13930,ORD,...,1437.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2018-02-20,DL,DL_CODESHARE,DL,5725,CP,N608CZ,5725,14057,PDX,...,129.0,0.0,0.0,32.0,0.0,0.0,None,NaN,NaN,None
4,2018-06-16,UA,UA_CODESHARE,UA,5853,OO,N136SY,5853,11503,EGE,...,120.0,0.0,0.0,24.0,0.0,0.0,None,NaN,NaN,None


In [5]:
flights.columns


Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [32]:
Delays = flights[["security_delay","nas_delay","carrier_delay","weather_delay","arr_delay"]]


In [33]:
Delays = Delays.fillna(0)

In [34]:
Delays = Delays.assign(total_expected = Delays['security_delay']+Delays['nas_delay']+Delays['carrier_delay']+Delays['weather_delay'])

In [35]:
Delays = Delays.mean()
Delays = Delays*100/Delays.total_expected

In [36]:
Delays

security_delay      0.000000
nas_delay          34.303302
carrier_delay      51.916620
weather_delay      13.780078
arr_delay          85.002798
total_expected    100.000000
dtype: float64

In [38]:
# In average and in order of priority: "carrier dalay", "national aviation system delay" and
# "weather delay" couse most of the delays but the aircrafts are compensae some of the delays during the flight
# i.e. going faster...